# Example usage for "movekit"

In [1]:
import movekit as mkit

In [2]:
print(mkit)

<module 'movekit' from '/Users/erencakmak/.conda/envs/mo_env/lib/python3.7/site-packages/movekit/__init__.py'>


### Read in CSV file:

In [3]:
# Enter absolute/complete path to CSV file-
path_to_file = "datasets/fish-5.csv"

In [4]:
# Read in CSV file using 'path_to_file' variable-
data = mkit.parse_csv(path_to_file)
print(data)



      time  animal_id       x       y
0        1        312  405.29  417.76
1        1        511  369.99  428.78
2        1        607  390.33  405.89
3        1        811  445.15  411.94
4        1        905  366.06  451.76
9        2        905  365.86  451.76
7        2        607  390.25  405.89
8        2        811  445.48  412.26
5        2        312  405.31  417.37
6        2        511  370.01  428.82
10       3        312  405.31  417.07
11       3        511  370.01  428.85
12       3        607  390.17  405.88
13       3        811  445.77  412.61
14       3        905  365.70  451.76
19       4        905  365.57  451.76
17       4        607  390.07  405.88
18       4        811  446.03  413.00
15       4        312  405.30  416.86
16       4        511  370.01  428.86
20       5        312  405.29  416.71
21       5        511  369.99  428.86
22       5        607  389.98  405.87
23       5        811  446.24  413.42
24       5        905  365.47  451.76
27       6

### Grouping data according to 'animal_id' attribute-
- 'grouping_data' function groups all values for each 'animal_id'.
- The input parameter is 'processed_data' which is the processed Pandas DataFrame
- The function returns a dictionary where-:
- key is animal_id, value in Pandas DataFrame for that 'animal_id'

In [ ]:
# To group data according to 'animal_id' attribute-
data_grouped = movekit.preprocess.grouping_data(data)

### Calculate absolute features: metric distance, direction, avg_speed, avg_acceleration 
- Calculate the metric distance and direction between two consecutive time frames/time stamps for each moving entity (animals)
- 'compute_average_speed()' function to compute average speed of an animal based on fps (frames per second) parameter
- Formula used-
- Average Speed = Total Distance Travelled / Total Time taken
- 'compute_average_speed()' function computes the average speed of an animal based on fps (frames per second) parameter

In [ ]:
data_features = movekit.feature_extraction.compute_absolute_features(data_grouped)
print(data_features)

### Compute derived features 
* For instance, compute the travelled distance for each animal, time moving etc. 

In [ ]:
distance = movekit.feature_extraction.compute_distance_summary(data_features)
stops = movekit.feature_extraction.compute_stop_summary(data_features)

### Compute euclidean distance matrix 

In [ ]:
distance_matrix = movekit.feature_extraction.distance_euclidean_matrix(data_features)
print(distance_matrix)

### Save to CSV 

In [ ]:
# save features to csv 
distance_matrix.to_csv("fish-5-features.csv")

### Using "tsfresh" Python library:

In [ ]:
# For extracting all time series related features, do-
extracted_features = movekit.feature_extraction.time_series_analyis(data_features)

In [ ]:
# Save to disk 
print(extracted_features)
#extracted_features.to_json("extraced_features_fish.json")